# VGG16 with Gabor filters

In [ ]:
from keras.applications.vgg16 import VGG16
weights = None  # 'imagenet'
input_shape = (224, 224, 3)  # (224, 224, 16)
classes = 10
model = VGG16(include_top=True, weights=weights, input_tensor=None, input_shape=input_shape, pooling=None, classes=classes)

In [ ]:
model.summary()

In [ ]:
from keras.layers import InputLayer
model.layers.pop(0)
gabor_input = InputLayer(input_shape=(224, 224, 32))
gabor_output = model(gabor_input)
gabor_model = Model(gabor_input, gabor_output)

# Run GaborNet

In [ ]:
import os

# Set parameters
project_root = os.path.realpath(os.pardir)
data_set = 'pixel'
stimulus_sets = ['jitter', 'static']  # 'static'
start_trial = 1
num_trials = 5
lambdas = [4, 8, 16, 32, 64]

epochs = 20
data_augmentation = 0
fresh_data = 0
n_gpus = 1

In [ ]:
import os
from tqdm import tqdm_notebook

for stimulus_set in tqdm_notebook(stimulus_sets, desc="Set"):
    for trial in tqdm_notebook(range(start_trial, num_trials+1), desc='Trial'):
        for lambd in tqdm_notebook(lambdas, desc='$\lambda$', leave=True):
            trial_label = f"{trial}_lambd={lambd}"
            args = (f"--data_set {data_set} --stimulus_set {stimulus_set} "
                    f"--trial_label {trial_label} --lambd {lambd} "
                    f"--data_augmentation {data_augmentation} --fresh_data {fresh_data} "
                    f"--n_gpus {n_gpus} --epochs {epochs}")  #  --model_name {model_name}
            os.system(f'python3 gabornet.py {args}')

# Plot performance

In [ ]:
# Load accuracy scores and plot
%matplotlib inline
import json
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns


# data_set = 'pixel'
stimulus_set = 'jitter'  # 'static'  # 'jitter'
noise_types = ['Original', 'Salt-and-pepper', 'Additive', 'Single-pixel']  # 'Original'
test_conditions = ['Same', 'Diff', 'NoPix']
results_dir = os.path.join(project_root, 'results', data_set, stimulus_set)

rows = []
test_rows = []

# for stimulus_set in stimulus_sets:
for trial in range(1, 1+num_trials):
    for noise_type in noise_types:
        for lambd in lambdas:

            trial_label = f"{trial}_lambd={lambd}"
            model_name = f"{noise_type}_{trial_label}"

            acc_scores = np.load(os.path.join(results_dir, f'{model_name}_ACC.npy'))
            valacc_scores = np.load(os.path.join(results_dir, f'{model_name}_VALACC.npy'))
            loss = np.load(os.path.join(results_dir, f'{model_name}_LOSS.npy'))
            valloss = np.load(os.path.join(results_dir, f'{model_name}_VALLOSS.npy'))

            with open(os.path.join(results_dir, f'{model_name}_CONDVALACC.json'), "r") as jf:
                cond_acc = json.load(jf)
            with open(os.path.join(results_dir, f'{model_name}_CONDVALLOSS.json'), "r") as jf:
                cond_loss = json.load(jf)

            for condition in test_conditions:
                test_rows.append({'Trial': trial, 'Noise Type': noise_type,
                                 'Condition': condition, 'Lambda': lambd,
                                  'Loss': cond_loss[condition], 'Accuracy': cond_acc[condition]})
            for epoch in range(epochs):
                rows.append({'Trial': trial, 'Noise Type': noise_type, 'Lambda': lambd, 
                             'Evaluation': 'Testing', 'Epoch': epoch+1, 'Loss': valloss[epoch], 
                             'Accuracy': valacc_scores[epoch]})

                rows.append({'Trial': trial, 'Noise Type': noise_type, 'Lambda': lambd, 
                             'Evaluation': 'Training', 'Epoch': epoch+1, 'Loss': loss[epoch], 
                             'Accuracy': acc_scores[epoch]})

scores = pd.DataFrame(rows, columns=['Trial', 'Noise Type', 'Evaluation', 'Lambda', 'Epoch', 'Loss', 'Accuracy'])
test_scores = pd.DataFrame(test_rows, columns=['Trial', 'Noise Type', 'Condition', 'Lambda', 'Loss', 'Accuracy'])
# scores
# test_scores

In [ ]:
test_scores

In [ ]:
# fig = plt.figure()
# g = sns.relplot(x='Epoch', y='Accuracy', style='Retina Out Width', hue='Evaluation', row='VVS Layers', col='Noise Type', kind='line', data=scores)
g = sns.relplot(x='Epoch', y='Accuracy', hue='Evaluation', row='Lambda', col='Noise Type', kind='line', data=scores)
# g = sns.relplot(x='Epoch', y='Loss', style='Retina Out Width', hue='Evaluation', row='VVS Layers', col='Noise Type', kind='line', data=scores)

In [ ]:
fig = plt.figure(figsize=(16,6))
# g = sns.catplot(x="Condition", y="Accuracy", hue="Retina Out Width", row="VVS Layers", col="Noise Type", kind="bar", data=test_scores)
g = sns.catplot(x="Condition", y="Accuracy", row='Lambda', col="Noise Type", kind="bar", data=test_scores)
# g = sns.catplot(x="Condition", y="Loss", hue="Retina Out Width", row="VVS Layers", col="Noise Type", kind="bar", data=test_scores)

In [ ]:
# from keras.layers import InputLayer
model.layers.pop(1)  # First convolutional layer
gabor_input = InputLayer(input_shape=(224, 224, 32))
gabor_output = model(gabor_input)
gabor_model = Model(gabor_input, gabor_output)

In [ ]:
from matplotlib import pyplot as plt
# import cv2
# from keras import backend as K
# import numpy as np

image_path = "/workspace/data/Lenna.png"
# img = image.load_img(image_path)
img = plt.imread(image_path)
print(img.shape)
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
print(type(img))
img = K.expand_dims(img, 0)
img = K.expand_dims(img, -1)

gabor_filter(img, kernel_tensor=gft)

In [ ]:
import tensorflow as tf
with tf.Session().as_default():
    plt.imshow(gabor_filter(img, kernel_tensor=gft).eval()[0,:,:,-1])

In [ ]:
type(gabor_filters)

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='vgg.png')